# Open session and import packages

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('project_churn').getOrCreate()

In [2]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Suppress all warnings
warnings.filterwarnings("ignore")

# Import data

In [3]:
# import train set
train = spark.read.csv('train.csv', header = True)
# import test set
test = spark.read.csv('test.csv', header = True)

# Data Format

In [4]:
train.show(truncate = False)

+---+----------+--------------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+------+
|id |CustomerId|Surname       |CreditScore|Geography|Gender|Age |Tenure|Balance  |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---+----------+--------------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+------+
|0  |15674932  |Okwudilichukwu|668        |France   |Male  |33.0|3     |0.0      |2            |1.0      |0.0           |181449.97      |0     |
|1  |15749177  |Okwudiliolisa |627        |France   |Male  |33.0|1     |0.0      |2            |1.0      |1.0           |49503.5        |0     |
|2  |15694510  |Hsueh         |678        |France   |Male  |40.0|10    |0.0      |2            |1.0      |0.0           |184866.69      |0     |
|3  |15741417  |Kao           |581        |France   |Male  |34.0|2     |148882.54|1            |1.0      |1.0           |84560.88 

In [5]:
# visualize all features type
train.printSchema()

root
 |-- id: string (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: string (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Tenure: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- NumOfProducts: string (nullable = true)
 |-- HasCrCard: string (nullable = true)
 |-- IsActiveMember: string (nullable = true)
 |-- EstimatedSalary: string (nullable = true)
 |-- Exited: string (nullable = true)



In [6]:
# removing features that don't serve any purpose
train = train.drop('id','surname')

In [7]:
# rename all to lower case letters, feature and data, and removing all blank spaces
for col in train.columns:
    train = train.withColumnRenamed(col, col.lower())
    train = train.withColumn(col, F.lower(col))
    train = train.withColumn(col, F.trim(col))

In [8]:
# visualize how the feature data are arranged
train.show(5, truncate = False)

+----------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+------+
|CustomerId|CreditScore|Geography|Gender|Age |Tenure|Balance  |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+----------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+------+
|15674932  |668        |france   |male  |33.0|3     |0.0      |2            |1.0      |0.0           |181449.97      |0     |
|15749177  |627        |france   |male  |33.0|1     |0.0      |2            |1.0      |1.0           |49503.5        |0     |
|15694510  |678        |france   |male  |40.0|10    |0.0      |2            |1.0      |0.0           |184866.69      |0     |
|15741417  |581        |france   |male  |34.0|2     |148882.54|1            |1.0      |1.0           |84560.88       |0     |
|15766172  |716        |spain    |male  |33.0|5     |0.0      |2            |1.0      |1.0           |15068.83       |

In [9]:
# changing data types to numeric type
int_columns = ['tenure', 'balance', 'estimatedsalary']

for col in int_columns:
    train = train.withColumn(col, F.col(col).cast(FloatType()))

int_columns = ['creditscore', 'Age', 'numofproducts', 'hascrcard', 'isactivemember', 'exited']
for col in int_columns:
    train = train.withColumn(col, F.col(col).cast(IntegerType()))
    

In [10]:
train.printSchema()

root
 |-- CustomerId: string (nullable = true)
 |-- creditscore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- tenure: float (nullable = true)
 |-- balance: float (nullable = true)
 |-- numofproducts: integer (nullable = true)
 |-- hascrcard: integer (nullable = true)
 |-- isactivemember: integer (nullable = true)
 |-- estimatedsalary: float (nullable = true)
 |-- exited: integer (nullable = true)



# Exploratory Data Analysis

In [11]:
train.summary().show()

+-------+--------------------+----------------+---------+------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+
|summary|          CustomerId|     creditscore|Geography|Gender|              Age|            tenure|           balance|     numofproducts|         hascrcard|     isactivemember|   estimatedsalary|             exited|
+-------+--------------------+----------------+---------+------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+
|  count|              165034|          165034|   165034|165034|           165034|            165034|            165034|            165034|            165034|             165034|            165034|             165034|
|   mean|1.5692005019026382E7|656.454373038283|     NULL|  NULL|38.12588787764945| 5.020353381727402| 55478.08669040132|1.554455

In [ ]:
+-------+--------------------+----------------+---------+------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+
|summary|          CustomerId|     creditscore|Geography|Gender|              Age|            tenure|           balance|     numofproducts|         hascrcard|     isactivemember|   estimatedsalary|             exited|
+-------+--------------------+----------------+---------+------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+
|  count|              165034|          165034|   165034|165034|           165034|            165034|            165034|            165034|            165034|             165034|            165034|             165034|
|   mean|1.5692005019026382E7|656.454373038283|     NULL|  NULL|38.12588787764945| 5.020353381727402| 55478.08669040132|1.5544554455445545|0.7539537307463916|0.49777015645260975|112574.82270602613|0.21159882206090866|
| stddev|   71397.81679067112|80.1033404871783|     NULL|  NULL|8.867204591410792|2.8061585665860913|62817.663267958495|0.5471536788441764|0.4307071240449495|0.49999654260421705| 50292.86554962783| 0.4084431067117287|
|    min|            15565701|           350.0|   france|female|             18.0|               0.0|               0.0|               1.0|               0.0|                0.0|             11.58|                0.0|
|    25%|         1.5633112E7|           597.0|     NULL|  NULL|             32.0|               3.0|               0.0|               1.0|               1.0|                0.0|          74637.57|                0.0|
|    50%|         1.5690164E7|           659.0|     NULL|  NULL|             37.0|               5.0|               0.0|               2.0|               1.0|                0.0|          117946.3|                0.0|
|    75%|         1.5756821E7|           710.0|     NULL|  NULL|             42.0|               7.0|         119919.12|               2.0|               1.0|                1.0|         155061.97|                0.0|
|    max|            15815690|           850.0|    spain|  male|             92.0|              10.0|          250898.1|               4.0|               1.0|                1.0|         199992.48|                1.0|
+-------+--------------------+----------------+---------+------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+

In [11]:
def eda_basic(df, cols:list):
    # lista auxiliar para adicionar os registros
    summary_data = []
    
    for feature in cols:
        # retirada de espaços vazios nos valores das colunas
        trimmed_col = df.withColumn(feature, F.trim(F.col(feature)))
        # cálculo de registros únicos
        unique_values = df.select(feature).distinct().count()
        # cálculo do total de registros
        total_values = df.count()
        # cálculo da porcentagem de missings nas features
        percent_missings = (df.select(feature).where(f'{feature} == "None" or {feature} == "" ').count()/total_values)*100
        # adiciona os valores calculados a um registro referente à respectiva feature
        summary_data.append((feature, unique_values, total_values, percent_missings))
        
    # criação do Dataframe de saída
    df = spark.createDataFrame(summary_data, ["feature", "unique_value", "total_values", "percent_missing"])
    
    return df

In [12]:
eda = eda_basic(train, train.columns[1:])

In [13]:
eda.show()

+---------------+------------+------------+---------------+
|        feature|unique_value|total_values|percent_missing|
+---------------+------------+------------+---------------+
|    creditscore|         457|      165034|            0.0|
|      Geography|           3|      165034|            0.0|
|         Gender|           2|      165034|            0.0|
|            Age|          69|      165034|            0.0|
|         tenure|          11|      165034|            0.0|
|        balance|       29807|      165034|            0.0|
|  numofproducts|           4|      165034|            0.0|
|      hascrcard|           2|      165034|            0.0|
| isactivemember|           2|      165034|            0.0|
|estimatedsalary|       54502|      165034|            0.0|
|         exited|           2|      165034|            0.0|
+---------------+------------+------------+---------------+



Train set dont have any missing data in features, so we need to looking for outliers and know if they are possible or not to consider and how we can work with him.

In [14]:
# visualizing percentile in each feature to know about outliers in data
percentile_data_pandas = train.select("age", "tenure", "balance", "numofproducts", "estimatedsalary").summary('95%', '96%', '97%', '98%', '99%', 'max').toPandas().T.reset_index()
percentile_data_pandas.columns = percentile_data_pandas.iloc[0]
percentile_data_pandas = percentile_data_pandas[1:]
percentile_data_pandas = percentile_data_pandas.reset_index(drop=True)

df_percentile = spark.createDataFrame(percentile_data_pandas)
df_percentile.show()

+---------------+---------+---------+---------+---------+---------+---------+
|        summary|      95%|      96%|      97%|      98%|      99%|      max|
+---------------+---------+---------+---------+---------+---------+---------+
|            age|       56|       57|       59|       62|       66|       92|
|         tenure|      9.0|      9.0|     10.0|     10.0|     10.0|     10.0|
|        balance|151657.81|153949.69|158623.03|164880.31|172274.02| 250898.1|
|  numofproducts|        2|        2|        2|        3|        3|        4|
|estimatedsalary|183487.98|184872.66|187756.23|190347.72|196111.34|199992.48|
+---------------+---------+---------+---------+---------+---------+---------+



# construct graphics about information to visualize how the data are distributed
distribution_feature = train.select("Age").groupBy("Age").count()

pd_distribution_feature = distribution_feature.toPandas()

plt.figure(figsize=(7, 3))
sns.barplot(data=pd_distribution_feature, x="Age", y="count", palette="viridis")
plt.xticks(np.arange(0, pd_distribution_feature["Age"].max()+5, 5))
plt.title("Age Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

# Data Wrangling

# Models

# Results

# Conclusion